# Collision analysis
This document tries to explain the processes of "outsourcing" the calculations of sightline studies without sharing the 3D model information

## Description of the process

Importing libraries

In [1]:
import pandas as pd
import numpy as np
import itertools
import numba
import os
import time
import multiprocessing as mp
from multiprocessing import Pool
from functools import partial

import os
os.environ["OMP_NUM_THREADS"] = "10"
os.environ["OPENBLAS_MAIN_FREE"] = "10"

Loger for python console 

In [2]:
def log(message):
    print('{} , {}'.format(time.time(), message))

## Reading points of view

In [3]:
pov_ = pd.read_csv(r".\pov_.csv",header=None )
pov_.columns = ["x","y","z" ]
print('{} Points of View'.format(len(pov_)))
pov_.head(10)

12 Points of View


,x,y,z
0,51.433256,-14.302818,1.322324
1,90.485795,-15.638943,1.322324
2,103.959348,-14.302818,1.322324
3,37.959702,-15.638943,1.322324
4,-11.158635,-15.638943,1.322324
5,2.314918,-14.302818,1.322324
6,51.433256,-14.302818,1.322324
7,90.485795,-15.638943,1.322324
8,103.959348,-14.302818,1.322324
9,37.959702,-15.638943,1.322324


## Reading targets (points over meshes)

In [4]:
target_ = pd.read_csv(r".\targets_.csv",header=None )
target_.columns = ["x1","y1","z1" ]
print('{} targets or points of interest'.format(len(target_)))
target_.head()

28800 targets or points of interest


,x1,y1,z1
0,44.573727,26.884630,0.000000
1,44.573727,25.813538,0.000000
2,44.573727,26.884630,0.514658
3,44.573727,24.742443,0.000000
4,44.573727,26.884630,1.543975


## Reading meshes bounding box

In [5]:
meshes_ = pd.read_csv(r".\context_.csv",header=None, index_col=0  )
meshes_.columns = ["xMax","yMax","zMax","xMin","yMin","zMin","id" ]
print('{} meshes in set'.format(len(meshes_)))
meshes_.head()

120 meshes in set


,xMax,yMax,zMax,xMin,yMin,zMin,id
0,,,,,,,
0,53.2885,10.9502,5.1466,44.5737,0.2393,0.0,7f2e5d7a-85e7-4aaf-8c3c-20758a8f439e
1,83.2885,10.9502,5.1466,74.5737,0.2393,0.0,0d9b2e4e-7dae-4432-93a1-de599bc947aa
2,65.9980,10.9502,5.1466,57.2832,0.2393,0.0,a58b6bc3-5dfb-4f8e-bae5-895d927b956b
3,53.2885,10.9502,5.1466,44.5737,0.2393,0.0,f05d1e0b-5a33-4c1d-a3f9-aaef0d6291b3
4,95.9980,10.9502,5.1466,87.2832,0.2393,0.0,f5f88bf8-c95a-4b45-a5f2-2665862dde8f


## Reading meshes faces

In [6]:
mesh_faces = pd.read_csv(r".\mesh_faces.csv",header=None  )
mesh_faces.columns = ["xMax","yMax","zMax","xMin","yMin","zMin", "id" ]
print('{} meshes faces in set'.format(len(mesh_faces)))
mesh_faces.head()

21888 meshes faces in set


,xMax,yMax,zMax,xMin,yMin,zMin,id
0,44.5737,125.9106,0.5147,44.5737,124.8395,0.0,a0c92873-dccd-477b-8ab7-41990f14e32b
1,44.5737,125.9106,1.5440,44.5737,123.7684,0.0,a0c92873-dccd-477b-8ab7-41990f14e32b
2,44.5737,123.7684,5.1466,44.5737,122.6974,0.0,a0c92873-dccd-477b-8ab7-41990f14e32b
3,44.5737,122.6974,5.1466,44.5737,121.6263,0.0,a0c92873-dccd-477b-8ab7-41990f14e32b
4,44.5737,121.6263,5.1466,44.5737,120.5552,0.0,a0c92873-dccd-477b-8ab7-41990f14e32b


## Creating all cross product of points vs targets to represent the lines of view

In [7]:
lines = pov_
lines = lines.assign(foo=1).merge(target_.assign(foo=1)).drop('foo', 1)
lines = lines.drop_duplicates()
lines = lines.reset_index()
lines = lines.drop(['index'], axis=1)

In [8]:
print('{} lines between POV and targets'.format(len(lines)))
lines.head()

50112 lines between POV and targets


,x,y,z,x1,y1,z1
0,51.433256,-14.302818,1.322324,44.573727,26.884630,0.000000
1,51.433256,-14.302818,1.322324,44.573727,25.813538,0.000000
2,51.433256,-14.302818,1.322324,44.573727,26.884630,0.514658
3,51.433256,-14.302818,1.322324,44.573727,24.742443,0.000000
4,51.433256,-14.302818,1.322324,44.573727,26.884630,1.543975


## Converting lines to bounding boxes

In [9]:
bbx = pd.DataFrame(columns = ["xMax","yMax","zMax","xMin","yMin","zMin"])
bbx['xMax'] = lines[['x', 'x1']].values.max(1)
bbx['yMax'] = lines[['y', 'y1']].values.max(1)
bbx['zMax'] = lines[['z', 'z1']].values.max(1)
bbx['xMin'] = lines[['x', 'x1']].values.min(1)
bbx['yMin'] = lines[['y', 'y1']].values.min(1)
bbx['zMin'] = lines[['z', 'z1']].values.min(1)

bbx.head()

,xMax,yMax,zMax,xMin,yMin,zMin
0,51.433256,26.884630,1.322324,44.573727,-14.302818,0.000000
1,51.433256,25.813538,1.322324,44.573727,-14.302818,0.000000
2,51.433256,26.884630,1.322324,44.573727,-14.302818,0.514658
3,51.433256,24.742443,1.322324,44.573727,-14.302818,0.000000
4,51.433256,26.884630,1.543975,44.573727,-14.302818,1.322324


## Finding if lines bounding box versus meshes bounding box intersect

### Estimation of total calculation in meshes versus lines bounding boxes (worst case scenario)

In [10]:
print('{} possible calculations'.format(len(bbx)* len(meshes_)))

6013440 possible calculations


In [11]:
class BoundingBox():
    #Bounding box defined by tuple of max & min points
    def __init__(self, point):
        self.XMax = point[0]
        self.YMax = point[1]
        self.ZMax = point[2]
        self.XMin = point[3]
        self.YMin = point[4]
        self.ZMin = point[5]

In [12]:
def checkmesh(lines, meshes):
    #iterate over points
    aa = BoundingBox(meshes)
    for b in lines:
        bb = BoundingBox(b)
        if  bb_intersection_over_union(aa,bb):
            return True
    return False 

In [13]:
def bb_intersection_over_union(boxA, boxB):
    interArea =  ((boxA.XMax > boxB.XMin) 
                  and (boxB.XMax > boxA.XMin) 
                  and (boxA.YMax > boxB.YMin) 
                  and (boxA.YMin < boxB.YMax) 
                  and (boxA.ZMax > boxB.ZMin) 
                  and (boxA.ZMin < boxB.ZMax) )
    
    return interArea

In [14]:
#Saving for possible limit in process like head(1000)
bbx2 = bbx #.head(1000)

resultA = meshes_.apply(lambda x: checkmesh( bbx2.values, x), axis=1)
meshes_['hits'] = resultA
print("Count of meshes with intersections")
print(len(meshes_[resultA]))

Count of meshes with intersections
120


## Finding mesh intersection

Filtering only the mesh faces that belong to a mesh from previews test

In [15]:
filter = mesh_faces["id"].isin(meshes_[resultA]['id'])
mesh_faces_filter = mesh_faces[filter]
print('{} faces to test'.format(len(mesh_faces)))

21888 faces to test


In [16]:

def checkFaces(Faces, line):
    #Face v line iterator
    for f in Faces:
        a = np.array(f[:-1],  dtype=np.float32)
        b = np.array(line,  dtype=np.float32)
        if intersection(a,b):
            return True
    return False

In [17]:
@numba.jit(['float32(float32,float32,float32)'], forceobj=True, parallel=True)
def tt(a,b,c):
    return (a - b) / c

@numba.jit(['float32(float32,float32,float32)'], forceobj=True, parallel=True)
def length(a,b,c):
    return (a**2 + b**2 + c**2)

@numba.jit(forceobj=True, parallel=True)
def normalC(ray):
    return [ray[3]-ray[0], ray[4]-ray[1],ray[5]-ray[2]]

@numba.jit(forceobj=True, parallel=True)
def intersection(aabb, ray):
    
    #Bounding box v line intersection detector
    normal = normalC(ray)
    #TODO : check divided by zero!
    t1 = tt(aabb[3],ray[0], normal[0])
    t2 = tt(aabb[0] , ray[0], normal[0])
    t3 = tt(aabb[4] , ray[1], normal[1])
    t4 = tt(aabb[1], ray[1], normal[1])
    t5 = tt(aabb[5],ray[2], normal[2])
    t6 = tt(aabb[2],ray[2], normal[2])
    
    tmax = min(max(t1, t2), max(t3, t4), max(t5, t6))

        # if tmax < 0, ray (line) is intersecting AABB, but whole AABB is behing us
    if (tmax < 0):
        return False

        # if tmin > tmax, ray doesn't intersect AABB
    tmin = max(min(t1, t2), min(t3, t4), min(t5, t6))
    if (tmin > tmax):
        return 0
    t= None
    if (tmin < float(0)):
        t = tmax
    else:
        t = tmin
    if (t * t) > length(normal[0],normal[1],normal[2]):
        return 0
    return 1

Filter lines with positive intersections

In [18]:
# start= time.time()
# pool = Pool(processes=10)
# fun = partial(checkFaces,mesh_faces_filter.values)
# resultsB = pool.map(fun,lines.values)
# end = time.time()
# print('total time {}'.format(end-start))
start= time.time()
resultsB = [checkFaces( mesh_faces_filter.values,x) for x in lines.values]
end = time.time()
print('total time {}'.format(end-start))

KeyboardInterrupt: 

In [ ]:
lines['hits']= resultsB
positive = len(lines[lines['hits'] == False])
print('{} lines with clean sight from POV to targets'.format(positive))
negative = len(lines[lines['hits'] == True])
print('{} lines with possible context intersection'.format(negative))

## Saving lines with no intersection

In [ ]:
lines[ lines['hits'] == False].to_csv('positive.csv')

## Saving lines with possible intersection

In [ ]:
lines[ lines['hits'] == True].to_csv('negative.csv')

In [ ]:

print('total time {}'.format(end-start))

In [ ]:
print(np.show_config())